<a href="https://colab.research.google.com/github/iskra3138/GluonTS/blob/master/DeepAR_evaluations_data3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install mxnet and gluonts

In [1]:
pip install --upgrade mxnet==1.6 gluonts

     |████████████████████████████████| 68.7MB 56kB/s 
     |████████████████████████████████| 419kB 34.5MB/s 
     |████████████████████████████████| 194kB 43.1MB/s 
     |████████████████████████████████| 7.4MB 22.9MB/s 
  Created wheel for ujson: filename=ujson-1.35-cp36-cp36m-linux_x86_64.whl size=68021 sha256=de090889d13c12c9f8fff2738d17724350c763138d7454c8c1ce10a5b3720a55
  Stored in directory: /root/.cache/pip/wheels/28/77/e4/0311145b9c2e2f01470e744855131f9e34d6919687550f87d1
Successfully built ujson
  Found existing installation: graphviz 0.10.1
    Uninstalling graphviz-0.10.1:
      Successfully uninstalled graphviz-0.10.1


## Dataset Loading

In [0]:
# Third-party imports
%matplotlib inline
import mxnet as mx
from mxnet import gluon
#import numpy as np
#import pandas as pd
import matplotlib.pyplot as plt
import json
#import os

In [0]:
from gluonts.dataset.repository.datasets import get_dataset, dataset_recipes
#from gluonts.dataset.util import to_pandas

In [0]:
from gluonts.dataset.common import ListDataset
from gluonts.dataset.field_names import FieldName

In [0]:
from gluonts.model.deepar import DeepAREstimator
from gluonts.model.deep_factor import DeepFactorEstimator
from gluonts.distribution.gaussian import GaussianOutput
from gluonts.distribution.student_t import StudentTOutput
from gluonts.trainer import Trainer
from gluonts.evaluation.backtest import make_evaluation_predictions
from gluonts.evaluation import Evaluator

## Experiment of DeepAR

## electricity

In [6]:
elec_dataset = get_dataset("electricity", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/electricity/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/electricity/test/data.json


In [7]:
print (len(elec_dataset.train))
print (len(elec_dataset.test))

321
2247


In [8]:
elec_dataset.metadata

MetaData(freq='1H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='321')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=24)

In [9]:
elec_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='321')

In [0]:
car = [int(elec_dataset.metadata.feat_static_cat[0].cardinality)]

In [11]:
next(iter(elec_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
test_ds_24 = ListDataset([{FieldName.START: entry['start'],
                        FieldName.TARGET: entry['target'],
                        FieldName.FEAT_STATIC_CAT: entry['feat_static_cat'],
                        FieldName.ITEM_ID: entry['item_id'],
                        }
                       for entry in iter(elec_dataset.test) if len(entry['target']) == 21068
                       ],
                     freq=elec_dataset.metadata.freq)

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.trainer.html
deepar_trainer=Trainer(
    ctx = 'cpu',
    epochs=100, # (default: 100).
    batch_size=32,# (default: 32).
    num_batches_per_epoch=50, # (default: 50).
    learning_rate=0.001, # (default:  10^−3 ).
    learning_rate_decay_factor=0.5, # (default: 0.5).
    patience = 10, # (default: 10).
    minimum_learning_rate=5e-05, # (default:  5⋅10^−5 ).
    clip_gradient = 10.0, # (default: 10).
    weight_decay=1e-08, #  (default  10^−8 ).
    init='xavier', #  (default: “xavier”).
    hybridize=True, 
    )

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=elec_dataset.metadata.freq, 
    prediction_length=elec_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [15]:
DA_predictor = DA_estimator.train(elec_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:14<00:00,  3.47it/s, epoch=100/100, avg_epoch_loss=5.12]


In [16]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=elec_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(elec_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 2247/2247 [00:00<00:00, 43004.27it/s]


{
    "MSE": 1912557.187110687,
    "abs_error": 8325086.188199043,
    "abs_target_sum": 128632956.0,
    "abs_target_mean": 2385.272140631948,
    "seasonal_error": 189.4933819611677,
    "MASE": 0.7417883000095216,
    "MAPE": 0.09824577784100318,
    "sMAPE": 0.1081300020135565,
    "OWA": NaN,
    "MSIS": 9.05810031616469,
    "QuantileLoss[0.1]": 3504008.1698703105,
    "Coverage[0.1]": 0.10662364634327237,
    "QuantileLoss[0.5]": 8325086.192954697,
    "Coverage[0.5]": 0.4749109924343578,
    "QuantileLoss[0.9]": 5126640.8107496295,
    "Coverage[0.9]": 0.8609998516540562,
    "RMSE": 1382.9523444828774,
    "NRMSE": 0.5797880757188911,
    "ND": 0.06471969895645595,
    "wQuantileLoss[0.1]": 0.027240361092769342,
    "wQuantileLoss[0.5]": 0.06471969899342667,
    "wQuantileLoss[0.9]": 0.03985480059052386,
    "mean_wQuantileLoss": 0.04393828689223996,
    "MAE_Coverage": 0.023570934084952783
}


In [17]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds_24, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds_24))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 321/321 [00:00<00:00, 7732.66it/s]


{
    "MSE": 595788.6721290431,
    "abs_error": 939137.0627183914,
    "abs_target_sum": 17268997.0,
    "abs_target_mean": 2241.56243509865,
    "seasonal_error": 189.64815247902152,
    "MASE": 0.6680059843478862,
    "MAPE": 0.08375125150692402,
    "sMAPE": 0.10422752072424935,
    "OWA": NaN,
    "MSIS": 7.732299862313674,
    "QuantileLoss[0.1]": 388311.1071445913,
    "Coverage[0.1]": 0.1285046728971963,
    "QuantileLoss[0.5]": 939137.0620438568,
    "Coverage[0.5]": 0.550882658359294,
    "QuantileLoss[0.9]": 534382.8787106394,
    "Coverage[0.9]": 0.8952492211838002,
    "RMSE": 771.8734819444461,
    "NRMSE": 0.34434618900565056,
    "ND": 0.054382837794134276,
    "wQuantileLoss[0.1]": 0.022486025514081177,
    "wQuantileLoss[0.5]": 0.05438283775507384,
    "wQuantileLoss[0.9]": 0.03094463903784565,
    "mean_wQuantileLoss": 0.03593783410233355,
    "MAE_Coverage": 0.02804603669089668
}


## exchange_rate

In [18]:
exchange_rate_dataset = get_dataset("exchange_rate", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/exchange_rate/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/exchange_rate/test/data.json


In [19]:
print (len(exchange_rate_dataset.train))
print (len(exchange_rate_dataset.test))

8
40


In [20]:
exchange_rate_dataset.metadata

MetaData(freq='1B', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='8')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=30)

In [21]:
exchange_rate_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='8')

In [0]:
car = [int(exchange_rate_dataset.metadata.feat_static_cat[0].cardinality)]

In [23]:
next(iter(exchange_rate_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=exchange_rate_dataset.metadata.freq, 
    prediction_length=exchange_rate_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [25]:
DA_predictor = DA_estimator.train(exchange_rate_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:07<00:00,  6.55it/s, epoch=100/100, avg_epoch_loss=-4.28]


In [26]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=exchange_rate_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(exchange_rate_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 40/40 [00:00<00:00, 891.10it/s]


{
    "MSE": 0.00020810532928313327,
    "abs_error": 11.262549614533782,
    "abs_target_sum": 975.9766580164433,
    "abs_target_mean": 0.8133138816803692,
    "seasonal_error": 0.006515919403041841,
    "MASE": 1.9573757224747912,
    "MAPE": 0.01372800821313302,
    "sMAPE": 0.013550027371305284,
    "OWA": NaN,
    "MSIS": 26.174232519846974,
    "QuantileLoss[0.1]": 5.441269254870712,
    "Coverage[0.1]": 0.33249999999999996,
    "QuantileLoss[0.5]": 11.262549614533782,
    "Coverage[0.5]": 0.7658333333333333,
    "QuantileLoss[0.9]": 4.6876753276214,
    "Coverage[0.9]": 0.9758333333333333,
    "RMSE": 0.014425856275560674,
    "NRMSE": 0.01773713273620234,
    "ND": 0.011539773540715182,
    "wQuantileLoss[0.1]": 0.005575204294259912,
    "wQuantileLoss[0.5]": 0.011539773540715182,
    "wQuantileLoss[0.9]": 0.004803060902243855,
    "mean_wQuantileLoss": 0.007306012912406315,
    "MAE_Coverage": 0.19138888888888883
}


## m4_daily

In [27]:
m4_daily_dataset = get_dataset("m4_daily", regenerate=True)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_daily/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_daily/test/data.json


In [28]:
print (len(m4_daily_dataset.train))
print (len(m4_daily_dataset.test))

4227
4227


In [29]:
m4_daily_dataset.metadata

MetaData(freq='D', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='4227')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=14)

In [30]:
m4_daily_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='4227')

In [0]:
car = [int(m4_daily_dataset.metadata.feat_static_cat[0].cardinality)]

In [32]:
next(iter(m4_daily_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=m4_daily_dataset.metadata.freq, 
    prediction_length=m4_daily_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [34]:
DA_predictor = DA_estimator.train(m4_daily_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:04<00:00, 12.10it/s, epoch=100/100, avg_epoch_loss=5.76]


In [35]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_daily_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_daily_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 4227/4227 [00:00<00:00, 84759.54it/s]


{
    "MSE": 431001.2286895228,
    "abs_error": 11350001.996948242,
    "abs_target_sum": 383097653.5168457,
    "abs_target_mean": 6473.649895515986,
    "seasonal_error": 52.78075560259988,
    "MASE": 3.531566152346949,
    "MAPE": 0.047104963140492014,
    "sMAPE": 0.033313251980026784,
    "OWA": NaN,
    "MSIS": 50.74410955389073,
    "QuantileLoss[0.1]": 7086121.456289673,
    "Coverage[0.1]": 0.1333772685795394,
    "QuantileLoss[0.5]": 11350001.990943909,
    "Coverage[0.5]": 0.46304369867180234,
    "QuantileLoss[0.9]": 5893128.498846435,
    "Coverage[0.9]": 0.8435567271621219,
    "RMSE": 656.5068382656214,
    "NRMSE": 0.10141216297785194,
    "ND": 0.029626915990622626,
    "wQuantileLoss[0.1]": 0.0184969064447117,
    "wQuantileLoss[0.5]": 0.02962691597494951,
    "wQuantileLoss[0.9]": 0.01538283632057616,
    "mean_wQuantileLoss": 0.02116888624674579,
    "MAE_Coverage": 0.042258947581871716
}


## m4_hourly

In [36]:
m4_hourly_dataset = get_dataset("m4_hourly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_hourly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_hourly/test/data.json


In [37]:
print (len(m4_hourly_dataset.train))
print (len(m4_hourly_dataset.test))

414
414


In [38]:
m4_hourly_dataset.metadata

MetaData(freq='H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='414')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=48)

In [39]:
m4_hourly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='414')

In [0]:
car = [int(m4_hourly_dataset.metadata.feat_static_cat[0].cardinality)]

In [41]:
next(iter(m4_hourly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=m4_hourly_dataset.metadata.freq, 
    prediction_length=m4_hourly_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [43]:
DA_predictor = DA_estimator.train(m4_hourly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:08<00:00,  5.83it/s, epoch=93/100, avg_epoch_loss=3.48]


In [44]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_hourly_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_hourly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 414/414 [00:00<00:00, 8475.57it/s]


{
    "MSE": 7865128.880396717,
    "abs_error": 8222872.113427162,
    "abs_target_sum": 145558863.59960938,
    "abs_target_mean": 7324.822041043147,
    "seasonal_error": 336.9046924038302,
    "MASE": 2.479186306387674,
    "MAPE": 0.1453893329738713,
    "sMAPE": 0.13861463901036014,
    "OWA": NaN,
    "MSIS": 28.5772269609828,
    "QuantileLoss[0.1]": 3211106.7888639458,
    "Coverage[0.1]": 0.19001610305958128,
    "QuantileLoss[0.5]": 8222871.939461708,
    "Coverage[0.5]": 0.5716586151368757,
    "QuantileLoss[0.9]": 4576358.177824592,
    "Coverage[0.9]": 0.801781400966184,
    "RMSE": 2804.4837101321727,
    "NRMSE": 0.3828739721481041,
    "ND": 0.056491730631024445,
    "wQuantileLoss[0.1]": 0.022060537637177344,
    "wQuantileLoss[0.5]": 0.0564917294358691,
    "wQuantileLoss[0.9]": 0.031439914167046806,
    "mean_wQuantileLoss": 0.03666406041336442,
    "MAE_Coverage": 0.08663110574342432
}


## m4_monthly

In [45]:
m4_monthly_dataset = get_dataset("m4_monthly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_monthly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_monthly/test/data.json


In [46]:
print (len(m4_monthly_dataset.train))
print (len(m4_monthly_dataset.test))

48000
48000


In [47]:
m4_monthly_dataset.metadata

MetaData(freq='M', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='48000')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=18)

In [48]:
m4_monthly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='48000')

In [0]:
car = [int(m4_monthly_dataset.metadata.feat_static_cat[0].cardinality)]

In [50]:
next(iter(m4_monthly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=m4_monthly_dataset.metadata.freq, 
    prediction_length=m4_monthly_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [52]:
DA_predictor = DA_estimator.train(m4_monthly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:04<00:00, 11.14it/s, epoch=85/100, avg_epoch_loss=7.15]


In [53]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_monthly_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_monthly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 48000/48000 [00:00<00:00, 330537.23it/s]


{
    "MSE": 2008925.6344161406,
    "abs_error": 531383032.3910675,
    "abs_target_sum": 4157157295.381714,
    "abs_target_mean": 4811.5246474325495,
    "seasonal_error": 616.9975625355751,
    "MASE": 1.1351977816797807,
    "MAPE": 0.1736513444118491,
    "sMAPE": 0.14204870913498038,
    "OWA": NaN,
    "MSIS": 22.753421840117973,
    "QuantileLoss[0.1]": 394392267.32355046,
    "Coverage[0.1]": 0.283741898148131,
    "QuantileLoss[0.5]": 531383032.35466766,
    "Coverage[0.5]": 0.5517233796296437,
    "QuantileLoss[0.9]": 404737391.2456589,
    "Coverage[0.9]": 0.793762731481541,
    "RMSE": 1417.3657377036247,
    "NRMSE": 0.2945772580547701,
    "ND": 0.1278236531923855,
    "wQuantileLoss[0.1]": 0.09487066264288106,
    "wQuantileLoss[0.5]": 0.12782365318362957,
    "wQuantileLoss[0.9]": 0.09735917178195094,
    "mean_wQuantileLoss": 0.10668449586948718,
    "MAE_Coverage": 0.11390084876541125
}


## m4_quarterly

In [54]:
m4_quarterly_dataset = get_dataset("m4_quarterly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_quarterly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_quarterly/test/data.json


In [55]:
print (len(m4_quarterly_dataset.train))
print (len(m4_quarterly_dataset.test))

24000
24000


In [56]:
m4_quarterly_dataset.metadata

MetaData(freq='3M', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='24000')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=8)

In [57]:
m4_quarterly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='24000')

In [0]:
car = [int(m4_quarterly_dataset.metadata.feat_static_cat[0].cardinality)]

In [59]:
next(iter(m4_quarterly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=m4_quarterly_dataset.metadata.freq, 
    prediction_length=m4_quarterly_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [61]:
DA_predictor = DA_estimator.train(m4_quarterly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:02<00:00, 22.17it/s, epoch=90/100, avg_epoch_loss=7.04]


In [62]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_quarterly_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_quarterly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 24000/24000 [00:00<00:00, 234022.32it/s]


{
    "MSE": 2022200.4034742871,
    "abs_error": 121690331.99066162,
    "abs_target_sum": 1147074070.9031982,
    "abs_target_mean": 5974.344119287491,
    "seasonal_error": 473.4332698179725,
    "MASE": 1.4951248787902678,
    "MAPE": 0.13639981814935406,
    "sMAPE": 0.11372371491502696,
    "OWA": NaN,
    "MSIS": 18.293836231600395,
    "QuantileLoss[0.1]": 72796851.83511963,
    "Coverage[0.1]": 0.164265625,
    "QuantileLoss[0.5]": 121690331.9389267,
    "Coverage[0.5]": 0.5931510416666667,
    "QuantileLoss[0.9]": 74196518.73234558,
    "Coverage[0.9]": 0.888125,
    "RMSE": 1422.0409289026413,
    "NRMSE": 0.23802460998383804,
    "ND": 0.10608759719836007,
    "wQuantileLoss[0.1]": 0.06346307852447566,
    "wQuantileLoss[0.5]": 0.10608759715325844,
    "wQuantileLoss[0.9]": 0.06468328472800693,
    "mean_wQuantileLoss": 0.07807798680191368,
    "MAE_Coverage": 0.05643055555555554
}


## m4_weekly

In [63]:
m4_weekly_dataset = get_dataset("m4_weekly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_weekly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_weekly/test/data.json


In [64]:
print (len(m4_weekly_dataset.train))
print (len(m4_weekly_dataset.test))

359
359


In [65]:
m4_weekly_dataset.metadata

MetaData(freq='W', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='359')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=13)

In [66]:
m4_weekly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='359')

In [0]:
car = [int(m4_weekly_dataset.metadata.feat_static_cat[0].cardinality)]

In [68]:
next(iter(m4_weekly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=m4_weekly_dataset.metadata.freq, 
    prediction_length=m4_weekly_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [70]:
DA_predictor = DA_estimator.train(m4_weekly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:03<00:00, 16.13it/s, epoch=100/100, avg_epoch_loss=5.96]


In [71]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_weekly_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_weekly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 359/359 [00:00<00:00, 5446.83it/s]


{
    "MSE": 420105.16883960285,
    "abs_error": 1626893.140007019,
    "abs_target_sum": 25616889.502441406,
    "abs_target_mean": 5488.941397566194,
    "seasonal_error": 216.54813922503882,
    "MASE": 3.210091946499933,
    "MAPE": 0.08736022681496754,
    "sMAPE": 0.08828131199940811,
    "OWA": NaN,
    "MSIS": 46.411162149740164,
    "QuantileLoss[0.1]": 841922.4322738646,
    "Coverage[0.1]": 0.21019927148060885,
    "QuantileLoss[0.5]": 1626893.1342086792,
    "Coverage[0.5]": 0.5819584315406043,
    "QuantileLoss[0.9]": 789106.0914321898,
    "Coverage[0.9]": 0.9033640454253268,
    "RMSE": 648.155204283359,
    "NRMSE": 0.1180838266137715,
    "ND": 0.06350861371564916,
    "wQuantileLoss[0.1]": 0.03286591185060253,
    "wQuantileLoss[0.5]": 0.06350861348930083,
    "wQuantileLoss[0.9]": 0.030804133786695078,
    "mean_wQuantileLoss": 0.04239288637553281,
    "MAE_Coverage": 0.06517391614884664
}


## m4_yearly

In [72]:
m4_yearly_dataset = get_dataset("m4_yearly", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/m4_yearly/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/m4_yearly/test/data.json


In [73]:
print (len(m4_yearly_dataset.train))
print (len(m4_yearly_dataset.test))

23000
23000


In [74]:
m4_yearly_dataset.metadata

MetaData(freq='12M', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='23000')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=6)

In [75]:
m4_yearly_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='23000')

In [0]:
car = [int(m4_yearly_dataset.metadata.feat_static_cat[0].cardinality)]

In [77]:
next(iter(m4_yearly_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=m4_yearly_dataset.metadata.freq, 
    prediction_length=m4_yearly_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [79]:
DA_predictor = DA_estimator.train(m4_yearly_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:01<00:00, 26.42it/s, epoch=85/100, avg_epoch_loss=7.34]


In [80]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=m4_yearly_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(m4_yearly_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 23000/23000 [00:00<00:00, 199283.57it/s]


{
    "MSE": 3349190.1243125624,
    "abs_error": 124059189.58959961,
    "abs_target_sum": 860724679.6655273,
    "abs_target_mean": 6237.135359895143,
    "seasonal_error": 288.79964609563865,
    "MASE": 3.7718017047444383,
    "MAPE": 0.19149751770402446,
    "sMAPE": 0.15143803337859618,
    "OWA": NaN,
    "MSIS": 62.34286478331613,
    "QuantileLoss[0.1]": 69181097.4132721,
    "Coverage[0.1]": 0.2872898550724656,
    "QuantileLoss[0.5]": 124059189.64224243,
    "Coverage[0.5]": 0.6370434782608703,
    "QuantileLoss[0.9]": 88966259.81285399,
    "Coverage[0.9]": 0.8265724637681161,
    "RMSE": 1830.079267221112,
    "NRMSE": 0.2934166346602872,
    "ND": 0.14413341748003355,
    "wQuantileLoss[0.1]": 0.08037540812720216,
    "wQuantileLoss[0.5]": 0.1441334175411946,
    "wQuantileLoss[0.9]": 0.10336204121325505,
    "mean_wQuantileLoss": 0.1092902889605506,
    "MAE_Coverage": 0.13258695652173993
}


## traffic

In [81]:
traffic_dataset = get_dataset("traffic", regenerate=False)

saving time-series into /root/.mxnet/gluon-ts/datasets/traffic/train/data.json
saving time-series into /root/.mxnet/gluon-ts/datasets/traffic/test/data.json


In [82]:
print (len(traffic_dataset.train))
print (len(traffic_dataset.test))

862
6034


In [83]:
traffic_dataset.metadata

MetaData(freq='H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='862')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=24)

In [84]:
traffic_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='862')

In [0]:
car = [int(traffic_dataset.metadata.feat_static_cat[0].cardinality)]

In [86]:
next(iter(traffic_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=traffic_dataset.metadata.freq, 
    prediction_length=traffic_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [88]:
DA_predictor = DA_estimator.train(traffic_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:10<00:00,  4.55it/s, epoch=100/100, avg_epoch_loss=-3.99]


In [89]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=traffic_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(traffic_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 6034/6034 [00:00<00:00, 70009.88it/s]


{
    "MSE": 0.0005942145662065895,
    "abs_error": 1335.8736866991967,
    "abs_target_sum": 8672.5710073933,
    "abs_target_mean": 0.05988682885450042,
    "seasonal_error": 0.015220711169889612,
    "MASE": 0.5958012602042452,
    "MAPE": 0.25716588057006,
    "sMAPE": 0.17007426683993804,
    "OWA": NaN,
    "MSIS": 8.578666672181198,
    "QuantileLoss[0.1]": 613.152266332694,
    "Coverage[0.1]": 0.1505772842779802,
    "QuantileLoss[0.5]": 1335.8736864569846,
    "Coverage[0.5]": 0.6049400618716166,
    "QuantileLoss[0.9]": 909.8370082599401,
    "Coverage[0.9]": 0.9201400397746076,
    "RMSE": 0.02437651669551229,
    "NRMSE": 0.4070430370380252,
    "ND": 0.15403433255955756,
    "wQuantileLoss[0.1]": 0.07070017251054922,
    "wQuantileLoss[0.5]": 0.15403433253162904,
    "wQuantileLoss[0.9]": 0.10490972140606414,
    "mean_wQuantileLoss": 0.10988140881608081,
    "MAE_Coverage": 0.05855246197473479
}


## electricity2

In [0]:
elec_dataset = get_dataset("electricity", regenerate=False)

In [91]:
print (len(elec_dataset.train))
print (len(elec_dataset.test))

321
2247


In [92]:
elec_dataset.metadata

MetaData(freq='1H', target=None, feat_static_cat=[CategoricalFeatureInfo(name='feat_static_cat', cardinality='321')], feat_static_real=[], feat_dynamic_real=[], feat_dynamic_cat=[], prediction_length=24)

In [93]:
elec_dataset.metadata.feat_static_cat[0]

CategoricalFeatureInfo(name='feat_static_cat', cardinality='321')

In [0]:
car = [int(elec_dataset.metadata.feat_static_cat[0].cardinality)]

In [95]:
next(iter(elec_dataset.train)).keys()

dict_keys(['start', 'target', 'feat_static_cat', 'item_id', 'source'])

In [0]:
test_ds_24 = ListDataset([{FieldName.START: entry['start'],
                        FieldName.TARGET: entry['target'],
                        FieldName.FEAT_STATIC_CAT: entry['feat_static_cat'],
                        FieldName.ITEM_ID: entry['item_id'],
                        }
                       for entry in iter(elec_dataset.test) if len(entry['target']) == 21068
                       ],
                     freq=elec_dataset.metadata.freq)

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.trainer.html
deepar_trainer=Trainer(
    ctx = 'cpu',
    epochs=100, # (default: 100).
    batch_size=32,# (default: 32).
    num_batches_per_epoch=50, # (default: 50).
    learning_rate=0.001, # (default:  10^−3 ).
    learning_rate_decay_factor=0.5, # (default: 0.5).
    patience = 10, # (default: 10).
    minimum_learning_rate=5e-05, # (default:  5⋅10^−5 ).
    clip_gradient = 10.0, # (default: 10).
    weight_decay=1e-08, #  (default  10^−8 ).
    init='xavier', #  (default: “xavier”).
    hybridize=True, 
    )

In [0]:
# https://gluon-ts.mxnet.io/api/gluonts/gluonts.model.deepar.html
DA_estimator = DeepAREstimator(
    freq=elec_dataset.metadata.freq, 
    prediction_length=elec_dataset.metadata.prediction_length,
    trainer = deepar_trainer, # (default: Trainer())
    context_length=None, # (default: None, in which case context_length = prediction_length)
    num_layers=2, # (default: 2)
    num_cells=40, # (default: 40)
    cell_type='lstm', # (default: ‘lstm’)
    dropout_rate=0.1, # (default: 0.1)
    use_feat_dynamic_real=False, # (default: False)
    use_feat_static_cat=True, # (default: False)
    use_feat_static_real=False, # (default: False)
    cardinality=car, # (default: None)
    #embedding_dimension=[20], # (default: [min(50, (cat+1)//2) for cat in cardinality])
    distr_output = StudentTOutput(), # (default: StudentTOutput())
    scaling=True, # (default: true)
    lags_seq=None, # (default: None, in which case these are automatically determined based on freq)
    time_features=None, # (default: None, in which case these are automatically determined based on freq)
    num_parallel_samples=100, # (default: 100)
)

In [99]:
DA_predictor = DA_estimator.train(elec_dataset.train)

  0%|          | 0/50 [00:00<?, ?it/s]

learning rate from ``lr_scheduler`` has been overwritten by ``learning_rate`` in optimizer.


100%|██████████| 50/50 [00:14<00:00,  3.45it/s, epoch=100/100, avg_epoch_loss=5.18]


In [100]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=elec_dataset.test, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(elec_dataset.test))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 2247/2247 [00:00<00:00, 27980.79it/s]


{
    "MSE": 2554453.941065254,
    "abs_error": 9163152.158949852,
    "abs_target_sum": 128632956.0,
    "abs_target_mean": 2385.272140631948,
    "seasonal_error": 189.4933819611677,
    "MASE": 0.7575806789415037,
    "MAPE": 0.09543141284583642,
    "sMAPE": 0.10828758843711765,
    "OWA": NaN,
    "MSIS": 9.191368226427423,
    "QuantileLoss[0.1]": 3731279.226968693,
    "Coverage[0.1]": 0.10041165999109891,
    "QuantileLoss[0.5]": 9163152.094332078,
    "Coverage[0.5]": 0.4587783711615484,
    "QuantileLoss[0.9]": 6181245.383898529,
    "Coverage[0.9]": 0.8512646491618454,
    "RMSE": 1598.26591688156,
    "NRMSE": 0.6700560031100349,
    "ND": 0.0712348720257183,
    "wQuantileLoss[0.1]": 0.029007179365206325,
    "wQuantileLoss[0.5]": 0.071234871523376,
    "wQuantileLoss[0.9]": 0.04805335721196152,
    "mean_wQuantileLoss": 0.04943180270018128,
    "MAE_Coverage": 0.030122879889235038
}


In [101]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds_24, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds_24))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 321/321 [00:00<00:00, 4640.98it/s]


{
    "MSE": 484715.10148394294,
    "abs_error": 937529.3135104179,
    "abs_target_sum": 17268997.0,
    "abs_target_mean": 2241.56243509865,
    "seasonal_error": 189.64815247902152,
    "MASE": 0.674031329476434,
    "MAPE": 0.08084996492337249,
    "sMAPE": 0.10280505898782312,
    "OWA": NaN,
    "MSIS": 7.593336734975212,
    "QuantileLoss[0.1]": 431103.8574109599,
    "Coverage[0.1]": 0.11500519210799581,
    "QuantileLoss[0.5]": 937529.312128529,
    "Coverage[0.5]": 0.494158878504673,
    "QuantileLoss[0.9]": 505048.93893814087,
    "Coverage[0.9]": 0.8816199376947039,
    "RMSE": 696.2148385979309,
    "NRMSE": 0.31059355193347127,
    "ND": 0.054289737470590675,
    "wQuantileLoss[0.1]": 0.024964035688405058,
    "wQuantileLoss[0.5]": 0.05428973739056929,
    "wQuantileLoss[0.9]": 0.029245991468881537,
    "mean_wQuantileLoss": 0.036166588182618635,
    "MAE_Coverage": 0.013075458636206303
}


In [102]:
forecast_it, ts_it = make_evaluation_predictions(
    dataset=test_ds_24, 
    predictor=DA_predictor, 
    num_samples=100, 
)

forecasts = list(forecast_it)
tss = list(ts_it)

evaluator = Evaluator(quantiles=[0.1, 0.5, 0.9])
agg_metrics, item_metrics = evaluator(iter(tss), iter(forecasts), num_series=len(test_ds_24))

print(json.dumps(agg_metrics, indent=4))

Running evaluation: 100%|██████████| 321/321 [00:00<00:00, 4824.36it/s]


{
    "MSE": 546099.5559470499,
    "abs_error": 942789.0771403313,
    "abs_target_sum": 17268997.0,
    "abs_target_mean": 2241.56243509865,
    "seasonal_error": 189.64815247902152,
    "MASE": 0.6729094831076863,
    "MAPE": 0.08056418550038828,
    "sMAPE": 0.10264765347231634,
    "OWA": NaN,
    "MSIS": 7.538680232482391,
    "QuantileLoss[0.1]": 430622.00082529336,
    "Coverage[0.1]": 0.1150051921079958,
    "QuantileLoss[0.5]": 942789.087135762,
    "Coverage[0.5]": 0.4957165109034267,
    "QuantileLoss[0.9]": 511843.51691199536,
    "Coverage[0.9]": 0.880581516095535,
    "RMSE": 738.9854910260755,
    "NRMSE": 0.3296742840863824,
    "ND": 0.0545943158795112,
    "wQuantileLoss[0.1]": 0.024936132702165235,
    "wQuantileLoss[0.5]": 0.05459431645831903,
    "wQuantileLoss[0.9]": 0.029639446744474815,
    "mean_wQuantileLoss": 0.036389965301653025,
    "MAE_Coverage": 0.01290238836967802
}
